In [ ]:
# Command Pattern
"""
Command
    - Command 패턴은 요청을 객체로 캡슐화하여 
        요청의 실행자(Invoker)와 요청의 수신자(Receiver)를 분리하는 디자인 패턴입니다.
"""

In [ ]:
"""
상황 설정
    주식 매수/매도 주문을 캡슐화(Command)
        주문이 체결되었는지 확인하고, 체결되지 않았으면 Undo를 처리하지 않음
    체결 여부는 Receiver (StockTrade)에서 확인
        체결되지 않은 주문은 "대기 중" 상태로 유지하며, 체결되면 Undo 가능
"""

In [ ]:
from abc import ABC, abstractmethod


# Command 인터페이스
class OrderCommand(ABC):
    @abstractmethod
    def execute(self):
        pass

    @abstractmethod
    def undo_command(self):
        pass


# Receiver: 주식 거래를 처리
class StockTrade:
    def __init__(self):
        self.executed_orders = set()

    def execute_order(self, order_id, stock, quantity):
        # 체결 여부를 가정 (여기선 50% 확률로 체결)
        import random
        is_executed = random.choice([True, False])

        if is_executed:
            self.executed_orders.add(order_id)
            print(f"[StockTrade] 체결 완료: {quantity}주 {stock} 매수/매도 완료.")
        else:
            print(f"[StockTrade] 체결 대기 중: {quantity}주 {stock}.")
        return is_executed

    def undo_order(self, order_id, stock, quantity):
        if order_id in self.executed_orders:
            self.executed_orders.remove(order_id)
            print(f"[StockTrade] 취소 완료: {quantity}주 {stock} 매수/매도 취소.")
        else:
            print(f"[StockTrade] 취소 불가: {quantity}주 {stock}은 체결되지 않았습니다.")


# Concrete Command 1: 매수 명령
class BuyStockCommand(OrderCommand):
    def __init__(self, stock_trade, order_id, stock, quantity):
        self.stock_trade = stock_trade
        self.order_id = order_id
        self.stock = stock
        self.quantity = quantity
        self.executed = False

    def execute(self):
        print(f"[Command] 매수 요청: {self.quantity}주 {self.stock}")
        self.executed = self.stock_trade.execute_order(self.order_id, self.stock, self.quantity)

    def undo_command(self):
        if self.executed:
            print(f"[Command] 매수 취소 요청: {self.quantity}주 {self.stock}")
            self.stock_trade.undo_order(self.order_id, self.stock, self.quantity)
        else:
            print(f"[Command] 매수 취소 실패: {self.quantity}주 {self.stock}은 체결되지 않음.")


# Concrete Command 2: 매도 명령
class SellStockCommand(OrderCommand):
    def __init__(self, stock_trade, order_id, stock, quantity):
        self.stock_trade = stock_trade
        self.order_id = order_id
        self.stock = stock
        self.quantity = quantity
        self.executed = False

    def execute(self):
        print(f"[Command] 매도 요청: {self.quantity}주 {self.stock}")
        self.executed = self.stock_trade.execute_order(self.order_id, self.stock, self.quantity)

    def undo_command(self):
        if self.executed:
            print(f"[Command] 매도 취소 요청: {self.quantity}주 {self.stock}")
            self.stock_trade.undo_order(self.order_id, self.stock, self.quantity)
        else:
            print(f"[Command] 매도 취소 실패: {self.quantity}주 {self.stock}은 체결되지 않음.")


# Invoker: 명령 실행 관리
class Broker:
    def __init__(self):
        self.history = []  # 명령 기록

    def execute_command(self, command):
        command.execute()
        self.history.append(command)


# 클라이언트 코드
def main():
    # Receiver 생성
    stock_trade = StockTrade()

    # Command 생성
    buy_command = BuyStockCommand(stock_trade, "order1", "AAPL", 100)
    sell_command = SellStockCommand(stock_trade, "order2", "GOOGL", 50)

    # Invoker 생성
    broker = Broker()

    # 명령 실행
    broker.execute_command(buy_command)
    broker.execute_command(sell_command)

    # Undo 명령 호출
    print("\n[Client] 명령 취소 요청:")
    buy_command.undo_command()
    sell_command.undo_command()


main()
